<h2>Fetching Data from website using API call (Air Quality Monitors -> Bristol Open Data Bristol)</h2>


In [ ]:
pip install scikit-learn

In [1]:
import numpy as np
import pandas as pd
import json
from sklearn import preprocessing

In [7]:
import requests

url = "https://maps2.bristol.gov.uk/server2/rest/services/ext/air_quality/MapServer/0/query?outFields=*&where=1%3D1&f=geojson"

def url_to_json(url):
    global data
    
    try:
    # https://www.geeksforgeeks.org/python/get-post-requests-using-python/

        response = requests.get(url)
        response.raise_for_status()  # Raises an HTTPError for bad responses (4xx or 5xx)

    except requests.exceptions.HTTPError as e:
        print("HTTP error occurred:", e)

    except requests.exceptions.RequestException as e:
        print("A request error occurred:", e)

        # https://www.geeksforgeeks.org/python/response-raise_for_status-python-requests/

    data = response.json()
    
    # data

    # Saving JSON data to a file => https://www.geeksforgeeks.org/python/reading-and-writing-json-to-a-file-in-python/
    # with open('Air_Quality_Bristol.json', 'w') as file:
    #     file.write(response.text)


# Reading JSON data from a URL and storing it in JSOn file.
url_to_json(url)

In [8]:
# Function to round of the coordinates up to 4 decimal places
def standardize_coordinates(coordinate):
    return round(coordinate, 4)

In [9]:
'''Trying to access the specific data from the JSON content'''

# print(data['type']) # o/p -> FeatureCollection

# print(data['features'][0]['type']) # o/p -> Feature

# print(data['features'][0]['id'])

id = data['features'][0]['id']

location = data['features'][0]["properties"]["location"]

pollutants = data['features'][0]['properties']['pollutants']

x = data['features'][150]['geometry']['coordinates'][0]
y = data['features'][150]['geometry']['coordinates'][1]

location_type = data['features'][0]['properties']['LocationClass']
# location_type

# scaler = preprocessing.MinMaxScaler()
# normalized_coordinates = scaler.fit_transform(x)

file = open('Air_Quality_Bristol.json')

file_data_in_dictionary = json.load(file)

temp_list = []

# Extracting the required data from the JSON file and storing it in a list of dictionaries

for i in file_data_in_dictionary['features']:
    temp_list.append({
        'ID' : i['id'],
        'Location' : i['properties']['location'],
        'Longitude' : i['geometry']['coordinates'][1],
        'Latitude' : i['geometry']['coordinates'][0],
        'Pollutants' : i['properties']['pollutants'],
        'Location Type' : i['properties']['LocationClass']
    })
    


pd_dataframe = pd.DataFrame(temp_list)   

# Normalizing the Longitude and Latitude columns and also rounding them off to 4 decimal places using the standardize_coordinates function
pd_dataframe['Normalized Longitude'] = standardize_coordinates((pd_dataframe['Longitude'] - pd_dataframe['Longitude'].min()) / (pd_dataframe['Longitude'].max() - pd_dataframe['Longitude'].min()))
pd_dataframe['Normalized Latitude'] = standardize_coordinates((pd_dataframe['Latitude'] - pd_dataframe['Latitude'].min()) / (pd_dataframe['Latitude'].max() - pd_dataframe['Latitude'].min()))


print(pd_dataframe.head())


# Saving the DataFrame to a CSV file
pd_dataframe.to_csv('Air_Quality_Bristol.csv', index=False)

# https://stackoverflow.com/questions/28056171/how-to-build-and-fill-pandas-dataframe-from-for-loop
# https://www.geeksforgeeks.org/python/normalize-a-column-in-pandas/

   ID           Location  Longitude  Latitude Pollutants  Location Type  \
0   1   Withywood School  51.407746 -2.627749    BTX NO2  Urban Traffic   
1   2     Colston Avenue  51.454557 -2.596814        NO2  Urban Traffic   
2   3      Blackboy Hill  51.469205 -2.613994        NO2  Urban Traffic   
3   4        Three Lamps  51.444210 -2.578333        NO2  Urban Traffic   
4   5  Bedminster Parade  51.442812 -2.595294        NO2  Urban Traffic   

   Normalized Longitude  Normalized Latitude  
0                0.0909               0.4508  
1                0.2938               0.5159  
2                0.3573               0.4797  
3                0.2490               0.5548  
4                0.2429               0.5191  


<h3>I faced issues installing PostgreSQL natively on my Windows machine, so I’m running it via Docker and using alternative libraries to connect from my Python environment.</h3>


In [11]:
pip install sqlalchemy

In [3]:
pip install pg8000

Note: you may need to restart the kernel to use updated packages.


In [10]:
import pandas as pd
from sqlalchemy import create_engine

# Sample DataFrame
# df = pd.DataFrame({
#     'Name': ['Shushant', 'Jose'],
#     'Age': [25, 30]
# })

# pd_dataframe is the transformed DataFrame which I want to insert into PostgreSQL
df = pd_dataframe 

# Deleting the Latitude and Longitude columns because we have normalized them and also renameing the normalized columns
df.drop(columns=['Longitude', 'Latitude'], inplace=True)
df.rename(columns={'Normalized Longitude': 'Longitude', 'Normalized Latitude': 'Latitude'}, inplace=True)

# PostgreSQL connection using pg8000
user = 'postgres'
password = 'shushant'
host = 'localhost'
port = '5432'
database = 'ETL Assignment'

engine = create_engine(f"postgresql+pg8000://{user}:{password}@{host}:{port}/{database}")

# Creating a DataBase table
table_name = 'Bristol'

# Inserting data
df.to_sql(table_name, engine, if_exists='append', index=False)

print("Data inserted successfully.")


Data inserted successfully.
